## RAG Langchain Pipeline

### Import Packages

In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [ ]:
from datasets import Dataset
from rag_utils import create_chunks, convert_docx_to_txt, create_retriever, load_file, create_langchain_pipeline, read_questions_answers, ragas_evaulation

### Create chunks

In [ ]:
docx_file_path = "../data/Evaluation Sets/Robinson Advisory.docx"
txt_file_path = convert_docx_to_txt(docx_file_path)

chunks = create_chunks(txt_file_path)

### Create Retriver

In [ ]:
retriever = create_retriever(chunks)

### Create langchain pipeline

In [ ]:
template = load_file("../prompts/system_message.txt")
langchain_executor = create_langchain_pipeline(retriever, template)

In [ ]:
question = "Who are the parties to the Agreement and what are their defined names?"
relevant_docs = retriever.get_relevant_documents(question)

In [10]:
relevant_docs

[Document(page_content='Term: The term of this Agreement shall commence on the Effective Date and shall continue until terminated in accordance with the provisions herein (the "Term").', metadata={'source': '../data/Evaluation Sets/Robinson Advisory.txt'}),
 Document(page_content='Governing Law and Jurisdiction:  This Agreement shall be governed by the laws of the State of Israel, without giving effect to the rules respecting conflicts of laws. The parties consent to the exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to this Agreement.', metadata={'source': '../data/Evaluation Sets/Robinson Advisory.txt'}),
 Document(page_content='Whereas,\tAdvisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and\nWhereas, \tAdvisor has agreed to provide the Company with such services, sub

In [11]:
question = "What is the termination notice?"

In [ ]:
response = langchain_executor.invoke(question)

In [13]:
response

"The termination notice is a written notice that can be provided by either party to terminate the Agreement, with or without cause, upon fourteen (14) days' prior written notice. The Company may terminate the Agreement immediately and without prior notice if the Advisor refuses or is unable to perform the Services, or is in breach of any provision of the Agreement."

In [ ]:
qa_docx_file_path = "/home/martin/Desktop/10academy/week11/LegalAI-RAG-Contract-Advisor/Raptor Contract.txt"
qa_txt_file_path = convert_docx_to_txt(qa_docx_file_path)
questions, ground_truths = read_questions_answers(qa_txt_file_path)

In [25]:
ground_truths = [[x] for x in ground_truths]

In [ ]:
contexts = []
answers = []
for question in questions:
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])
    answers.append(langchain_executor.invoke(question))

In [ ]:
data = {
            "question": questions, # list 
            "answer": answers, # list
            "contexts": contexts, # list list
            "ground_truths": ground_truths # list Lists
        }


# Convert dict to dataset
dataset = Dataset.from_dict(data)

df = ragas_evaulation(dataset)


In [ ]:
df